### Rep Scraping

Grab Rep from Zip

In [11]:
# Pull rep from zipcode

import mechanize

# Insantiate browser
br = mechanize.Browser()
br.set_handle_robots(False)

# Open lookup link
br.open("https://ziplook.house.gov/")

# Find form and input zipcode
br.select_form(nr=1)
br.form["ZIP"] = "22182"
br.submit()

<response_seek_wrapper at 0x1d5bbef1c50 whose wrapped object = <closeable_response at 0x1d5bd2c5490 whose fp = <_io.BufferedReader name=1416>>>

Grab Rep Name from HTML

In [12]:
# Scrape for rep name

import requests
import lxml.html

# Open URL
URL = br.geturl()  
page = requests.get(URL, headers={
    "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"})

# Scrape HTML
html = lxml.html.fromstring(page.content)
name = html.xpath("/html/body/div[2]/div/div[2]/section/div/div[2]/div[2]/div/div/p/a[1]")
name[0].text

'Gerald E. Connolly '

Grab Rep Infos With BS

In [13]:
# BS Table Scraping
from bs4 import BeautifulSoup

soup: BeautifulSoup = BeautifulSoup(page.content, "html.parser")
table = soup.find('div', attrs = {'class':'col-xs-16 col-md-4 pull-md-12 repdistrict'}) 
if (len(name) > 1): 
    table = soup.find('div', attrs = {'id':'PossibleReps'}) 
table

<div class="col-xs-16 col-md-4 pull-md-12 repdistrict">
<h1>Find Your Representative</h1>
<p>
<strong>Information provided:</strong><br/>
<em>Zip code: 22182
</em><br/>

		is located in the 11th Congressional district of Virginia.

		<p>The representative for this district is:</p>
<div id="PossibleReps">
<div id="RepInfo" name="RepeatReps">
<p class="rep color4"><img alt="" border="0" class="repPhoto" src="/zip/pictures/va11_connolly.jpg"/><br/>
<a href="https://connolly.house.gov/">Gerald E. Connolly </a><br/>
                	Democrat<br/>
<a href="https://connolly.house.gov/"><img alt="Go to website" class="repLink" height="13" src="/content/static/img/icon-web.gif" width="13"/></a>
<a href="https://connolly.house.gov/contact/zip-auth.htm"><img alt="Go to contact form" class="repLink" height="11" src="/content/static/img/icon-email.gif" width="17"/></a>
</p>
</div>
</div>
</p></div>

In [14]:


info = table.findAll('div')[0].text.split()
info[-4]

'Gerald'

In [15]:
info = table.find('p')
district = str.splitlines(info.text)[5]
party = str.splitlines(info.text)[12]


In [16]:
import regex as re
party = re.sub(r'[\t ]', "", party)
party

'Democrat'

In [17]:
dist = re.match("([^0-9]*)([0-9].*)", district).group(2)
dist



'11th Congressional district of Virginia.'

In [18]:
import google.generativeai as genai

In [19]:
import config
genai.configure(api_key=config.api_key)

In [ ]:
generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,    
}

model = genai.GenerativeModel(model_name="gemini-2.0-flash", generation_config=generation_config)
convo = model.start_chat(history = [])
convo.send_message(f"Give me the following information about representative {name[0].text} in bullet points: 1. Their three most recent key votes ordered by date (most recent to least recent) sourced from the representative's Ballotopedia.org \"key votes\" section in a numbered list format containing the vote name, whether the vote passed, whether whether the representative voted yay or nay for it, and the date of the vote. 2. If the representative serves on a house committee, and if so which ones. If not, return that the representative is not on any house committees. Source must be from Ballotopedia or the representative's house.gov website. 3. List the house committee in a numbered list and the relevant subcommittees under them in bullet points.3. One relevant fact about the representative  (one sentence)")
print(convo.last.text)  

Okay, here's the information about Representative Gerald E. Connolly, formatted as requested:

*   **Three Most Recent Key Votes (Ballotpedia):**

1.  Providing for congressional disapproval under chapter 8 of title 5, United States Code, of the rule submitted by the Consumer Financial Protection Bureau relating to "Credit Card Late Fees"; Passed (219-200); Connolly voted Yay; May 8, 2024
2.  Providing for consideration of the bill (H.R. 8070) to reauthorize the FISA Amendments Act of 2008; Failed (204-216); Connolly voted Nay; April 17, 2024
3.  Reauthorizing the FISA Amendments Act of 2008; Passed (273-147); Connolly voted Nay; April 13, 2024

*   **House Committee Assignments (house.gov):**
    Yes, Representative Connolly serves on the following committees:
    1.  Committee on Oversight and Accountability
           *   Subcommittee on Cybersecurity, Information Technology, and Government Innovation
           *   Subcommittee on Government Operations
    2.  Committee on Foreign 